In [47]:
import os

from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

import PyPDF2

os.environ["OPENAI_API_KEY"] = 

In [48]:
llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo", openai_api_key=os.getenv("OPENAI_API_KEY"))

### 자소서 받아서 읽어오기

In [49]:
def extract_text_from_pdf(file_path):
    """
    pdf path를 input으로 받아 string 으로반환
    """
    with open(file_path, "rb") as file:
        pdf_reader = PyPDF2.PdfReader(file)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            text += pdf_reader.pages[page_num].extract_text()
    return text

In [50]:
file_path = "Resume.pdf"
pdf_text = extract_text_from_pdf(file_path)
print(pdf_text)

지원동기: 
 
해당 교육과정을  알게 된 것은, 1년 전 교내 취업진로상담관을  통해서  입니다 .  컴퓨터  공학과  함
께 철학을  전공한  제 강점을  살리고  싶었습니다 . 인문학을  통해 배운 사람에  대한 이해는  금융에  
있어서  도움이  될  역량이라  판단하였습니다 . 그래서  핀테크  전문가  과정에  지원하게  되었습니다 . 
다양한  취업아카데미  중, 핀테크  핵심 기술과  연관된  취업 아카데미는  서울대학교  핀테크  전문가  
양성과정이   유일합니다 . 실습 과정 중 특히 블록체인에  관심이  많습니다 .  ‘ 블록체인을  활용한  
스터디  관리 플랫폼‘을  주제로  한이음  공모전에  참여하려고  했으나 , 개발 능력 부족 및 팀원들과
의 교류 부족으로   프로젝트에  실패한  경험이  있습니다 .  실패의  결정적인  원인은   개발에  대한 
열정에  있었습니다 .  다른 대학의  사람들과 , 학업을  병행하며  프로젝트를  진행했기에   일정을  조
율하기가  쉽지 않았고  서로 간의 온도 차이가  있었습니다 . 6개월 동안 이어져  온 프로젝트를  매듭
짓지 못해 아쉬움이  있습니다 . 이번 핀테크  전문가  과정을  통해 제 모든 열정을  쏟아붓고  싶습니
다. 
프로그래밍  경험은  있지만 ,  빅데이터   프로젝트  경험은  부족하며 , 금융에  대한 이해 또한 부족합
니다.  하지만  그 누구보다  최선을  다해서  배울 자신이   자신이  있습니다 .  단 1번의 휴학 없이  
학교를  빠르게  졸업한  이유도 , 해당 교육과정에  참여하기  위해서였습니다 . 이번 해에 교육과정이  
열린 것에 대해서  정말 다행으로  생각하며 , 꼭 교육과정을  통해 뵙고 싶습니다 . 감사합니다 . 
 


### 자소서 기반 문항

In [51]:
def coverletter_interview(llm) -> LLMChain:
    """
    자기소개서를 기반으로 면접 10개를 뽑아주는 Chain
    coverletter_interview(llm).run(coverletter=text)
    """    
    role_template = """
        given the academic cover letter about a person {coverletter}. I want you to create:
        professional interview questions that are derived from their information on given cover letter.
        \
        write with korean.
        Write 10 possible interview questions as a numbered list.
        
        Questions:

     """

    interview_prompt_template = PromptTemplate(
        input_variables=["coverletter"],
        template=role_template,
    )
    
    return LLMChain(llm=llm, prompt=interview_prompt_template)

In [52]:
cl = coverletter_interview(llm)
cl_interviews = cl.run(
        coverletter=pdf_text
    )

print(cl_interviews)

1. 지원동기에 대해 자세히 설명해주세요.
2. 컴퓨터 공학과 철학을 함께 전공한 경험에 대해 이야기해주세요.
3. 인문학을 배워 금융에 도움이 될 역량이 있다고 생각하시나요? 그 이유는 무엇인가요?
4. 핀테크 전문가 과정을 선택한 이유는 무엇인가요?
5. 서울대학교 핀테크 전문가 양성과정을 선택한 이유는 무엇인가요?
6. 블록체인에 대한 관심이 많다고 하셨는데, 블록체인 기술에 대해 어떤 부분이 특히 매력적인가요?
7. 이전에 참여했던 이음 공모전에서 프로젝트가 실패한 이유는 무엇인가요?
8. 프로젝트 실패 경험을 통해 얻은 교훈은 무엇인가요?
9. 프로그래밍 경험은 있지만 빅데이터 프로젝트 경험이 부족하다고 하셨는데, 이를 보완하기 위해 어떤 노력을 하고 있나요?
10. 이번 핀테크 전문가 과정을 통해 얻고자 하는 목표는 무엇인가요?


### 직무 기반 문항

In [53]:
def tech_interview(llm) -> LLMChain:
    """
    지원하고자 하는 직무를 기반으로 테크 면접 질문 10개 생성.
    tech_interview(llm).run(job=직무)
    """ 
    
    role_template = """
        based on the job a person wants to work for I want you to
        Create: professional or technical interview questions for the {job}.
        \
        write with korean.
        Write 10 possible interview questions as a numbered list.
        
        Questions:
     """

    interview_prompt_template = PromptTemplate(
        input_variables=["job"],
        template=role_template,
    )
    return LLMChain(llm=llm, prompt=interview_prompt_template)

In [54]:
jb = tech_interview(llm)
jb_interviews = jb.run(
        job="데이터분석"
    )

In [22]:
print(jb_interviews)

1. 데이터 분석 경험이 어떻게 되시나요?
2. 어떤 데이터 분석 도구를 사용하시나요?
3. 데이터 분석을 통해 어떤 문제를 해결했나요?
4. 데이터 분석에서 가장 어려운 부분은 무엇인가요?
5. 데이터 분석을 위해 필요한 기술은 무엇인가요?
6. 데이터 분석에서 가장 중요한 요소는 무엇인가요?
7. 데이터 분석을 통해 얻은 결과를 어떻게 활용하시나요?
8. 데이터 분석에서 발생한 문제를 해결하기 위해 어떤 방법을 사용하시나요?
9. 데이터 분석에서 가장 성공적인 프로젝트는 무엇이었나요?
10. 데이터 분석에서 발생한 문제를 해결하기 위해 어떤 도구를 사용하시나요?


### 받은 interview 통합

In [34]:
interviews = "자기소개서 면접 질문:\n\n"+cl_interviews+"\n\n\n직무 면접 질문:\n\n"+jb_interviews

In [35]:
print(interviews)

자기소개서 면접 질문:

1. 지원동기를 알려주신 것처럼, 핀테크 전문가 과정에 지원하게 된 이유가 무엇인가요?
2. 컴퓨터 공학과 철학을 함께 전공하셨다고 하셨는데, 이 두 분야를 결합하여 어떤 분야에서 활용하고 싶으신가요?
3. 이전에 참여하셨던 '블록체인을 활용한 스터디 관리 플랫폼' 프로젝트에서 어떤 역할을 맡으셨나요?
4. 프로젝트 실패의 원인으로 개발에 대한 열정 부족을 꼽으셨는데, 이번 핀테크 전문가 과정에서는 어떻게 열정을 쏟아낼 계획인가요?
5. 빅데이터 프로젝트 경험이 부족하다고 하셨는데, 이를 보완하기 위해 어떤 노력을 하고 있나요?
6. 핀테크 분야에서 가장 관심 있는 기술이나 분야가 무엇인가요?
7. 이번 핀테크 전문가 과정에서 어떤 목표를 가지고 참여하고 있나요?
8. 금융에 대한 이해가 부족하다고 하셨는데, 이를 보완하기 위해 어떤 자기계발 방법을 사용하고 있나요?
9. 이번 핀테크 전문가 과정을 마치고 나서, 어떤 분야에서 활동하고 싶으신가요?
10. 핀테크 분야에서 성공적인 전문가가 되기 위해 필요한 역량이 무엇이라고 생각하시나요?


직무 면접 질문:

1. 데이터 분석 경험이 어떻게 되시나요?
2. 어떤 데이터 분석 도구를 사용하시나요?
3. 데이터 분석을 통해 어떤 문제를 해결했나요?
4. 데이터 분석에서 가장 어려운 부분은 무엇인가요?
5. 데이터 분석을 위해 필요한 기술은 무엇인가요?
6. 데이터 분석에서 가장 중요한 요소는 무엇인가요?
7. 데이터 분석을 통해 얻은 결과를 어떻게 활용하시나요?
8. 데이터 분석에서 발생한 문제를 해결하기 위해 어떤 방법을 사용하시나요?
9. 데이터 분석에서 가장 성공적인 프로젝트는 무엇이었나요?
10. 데이터 분석에서 발생한 문제를 해결하기 위해 어떤 도구를 사용하시나요?


### 사용

In [55]:
interviews = cl_interviews+jb_interviews
interviews_list = interviews.split("\n")

In [64]:
def ai_answer(llm) -> LLMChain:
    """
    질문과 답변을 토대로 질문을수정.
    ai_answer(llm).run(job=직무, question=질문,answer=답변)
    """
    
    role_template = """You're an interview expert. Based on the question about interviews and the interviewee response.
            interviews question: {question} interviewee response: {answer}.
            Please professionally edit the interviewee response, and feel free to use the user's information below if needed. 
            The user's information is the job position they want to work in.
            job position: {job}.
            \
            Write with korean.
            
            Edit response:
            """
    
        

    edit_template = PromptTemplate(
        input_variables=["job", "question", "answer"],
        template=role_template,
    )
    return LLMChain(llm=llm, prompt=edit_template)

example

In [78]:
job="데이터분석"
question = "전에 참여하셨던 '블록체인을 활용한 스터디 관리 플랫폼' 프로젝트에서 어떤 역할을 맡으셨나요?"
answer = "팀장을 맡아서 팀원들끼리 원만한 관계를 유지하도록 노력했고 플랫폼 구축 개발을 진행하였습니다."

In [79]:
ai = ai_answer(llm)
ai_edit = ai.run(
        job=job,
        question=question,
        answer=answer
    )

In [80]:
print(ai_edit)

저는 '블록체인을 활용한 스터디 관리 플랫폼' 프로젝트에서 팀장을 맡아서 팀원들과의 원활한 협업을 유지하며, 플랫폼 구축 및 개발을 주도적으로 진행하였습니다. 이 경험을 통해 데이터분석 분야에서도 팀원들과의 협업과 데이터 분석 업무를 효율적으로 수행할 수 있는 능력을 보유하고 있습니다.


### 피드백 메세지

In [ ]:
"Please provide the strengths and weaknesses of the interviewee's answers from the interviewer's perspective."

In [ ]:
provide the strengths and weaknesses of the corresponding interviewee response according to your judgment. 

In [83]:
def ai_feedback(llm) -> LLMChain:
    """
    면접 질문과 답변을 통해 피드백 생성.
    ai_feedback(llm).run(question=질문,answer=대답)
    """
    
    
    role_template = """Please review the interview records between the interviewer question and interviewee response
                    and provide feedback on the corresponding response. 
                    interviews question: {question} interviewee response: {answer}. 
                    provide the strengths and weaknesses of the corresponding interviewee from the interviewer's perspective. 
                    If there are no strengths or weaknesses to mention, please say 'no feedback'.

                    \
                    Write with korean.

                    Strengths:
                    Weakness:
                    """

    feedback_template = PromptTemplate(
        input_variables=["question", "answer"],
        template=role_template,
    )
    return LLMChain(llm=llm, prompt=feedback_template)

In [84]:
feedback = ai_feedback(llm)
ai_feedback = feedback.run(
        question=question,
        answer=answer
    )

In [85]:
print(ai_feedback)

강점:
- 팀장 역할을 맡아서 팀원들 간의 관계를 원만하게 유지하려는 노력을 보였다.
- 프로젝트의 핵심인 플랫폼 구축 개발을 진행했다.

약점:
- 역할에 대한 구체적인 설명이 부족하다. 팀장으로서 어떤 일을 했는지, 어떤 어려움을 겪었는지 등에 대한 내용이 없다. 
- 개발 과정에서 어떤 기술을 사용했는지, 어떤 문제를 해결했는지 등에 대한 정보가 없다. 더 구체적인 설명이 필요하다.


### 강점은 살리고 약점을 보완

In [86]:
def modify_response(llm) -> LLMChain:
    """
    피드백을 바탕으로 최종 모법 답안 완성
    modify_response(llm).run(question=질문, answer=대답, feedback=피드백)
    """
    
    role_template = """Please modify the interviewee response: {answer} to the interviewer's question {question} based on the feedback: {feedback},
                    while preserving the strengths and addressing the weaknesses.

                    \
                    Write with korean.

                    Modify interviewee response:
                    """

    modify_template = PromptTemplate(
        input_variables=["question", "answer", "feedback"],
        template=role_template,
    )
    return LLMChain(llm=llm, prompt=modify_template)

In [87]:
modify = modify_response(llm)
ai_feedback = modify.run(
        question=question,
        answer=answer,
        feedback=ai_feedback
    )

결론적으로는 좋아 보이지만 토크제한 때문에 자기소개서 안의 내용을 인식할 수 없기 때문에 사용자가 자기소개서의 적은 내용과 다른 부분이 포함되어 있다.

print(ai_feedback)